In [1]:
path = "/home/efectn/python-music-classification"

In [25]:
import tensorflow as tf
import numpy as np

model = tf.keras.models.load_model(path + "/models/model_dnn.h5")


TypeError: Unable to convert function return value to a Python type! The signature was
	() -> handle

In [19]:
import librosa
import math

n_fft=2048
hop_length=512
n_mfcc=13

genres = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']

audio, fs = librosa.load(path+"/blues.00002.wav", sr=22500)

duration = librosa.get_duration(y=audio, sr=fs)

print(f"Dosya süresi: {duration:.2f} saniye")

num_segments = int(duration / 2)
samples_per_track = fs * duration
samps_per_segment = int(samples_per_track/num_segments)
mfccs_per_segment = math.ceil(samps_per_segment/hop_length)

mfccs = []
expected_frames = 88

for seg in range(num_segments):
    start_sample = seg * samps_per_segment
    end_sample = start_sample + samps_per_segment

    mfcc = librosa.feature.mfcc(y=audio[start_sample:end_sample], sr=fs, n_fft=n_fft, hop_length=hop_length, n_mfcc=n_mfcc)
    mfcc = mfcc.T

    if mfcc.shape[0] > expected_frames:
        mfcc = mfcc[:expected_frames, :]
    elif mfcc.shape[0] < expected_frames:
        pad_width = expected_frames - mfcc.shape[0]
        mfcc = np.pad(mfcc, ((0, pad_width), (0, 0)), mode='constant')

    mfccs.append(mfcc)
        
X_train_cnn = np.array(mfccs)

def make_prediction(model, X):
    global genres
    preds_probs = []

    for X_current in X:
        X_current = X_current[np.newaxis, ...]  
        pred = model.predict(X_current, verbose=0)[0]
        preds_probs.append(pred)

    preds_probs = np.array(preds_probs)
    avg_probs = np.mean(preds_probs, axis=0)

    pred_idx = np.argmax(avg_probs)
    pred_genre = genres[pred_idx]
    avg_probs_percent = avg_probs * 100

    return pred_genre, pred_idx, avg_probs_percent


genre_name, genre_index, avg_probs = make_prediction(model, X_train_cnn)

print(f"\nGenel Tahmin: {genre_name.upper()} (Sınıf ID: {genre_index})")
print("Ortalama Olasılıklar:")
for i, genre in enumerate(genres):
    print(f"  {genre:10s}: {avg_probs[i]:5.2f}%")

/tmp/ipykernel_302474/865766874.py:10: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, fs = librosa.load(path+"/blues.00002.wav", sr=22500)


FileNotFoundError: [Errno 2] No such file or directory: '/home/efectn/python-music-classification/blues.00002.wav'

In [9]:
import math
import numpy as np
import librosa
import tensorflow as tf

model = tf.keras.models.load_model(path + "/models/model_dnn.h5")


def load_and_pad_audio(audio_file, sample_rate=22500, target_duration=30):
    signal, sr = librosa.load(audio_file, sr=sample_rate)
    total_duration = librosa.get_duration(y=signal, sr=sr)

    target_length = sample_rate * target_duration  # örn. 30 saniye
    full_segments = int(np.ceil(total_duration / target_duration))

    padded_signal = []

    for i in range(full_segments):
        start = int(i * target_length)
        end = int((i + 1) * target_length)
        segment = signal[start:end]

        if len(segment) < target_length:
            # Padding with zeros to reach 30s
            padding = np.zeros(int(target_length - len(segment)))
            segment = np.concatenate((segment, padding))

        padded_signal.append(segment)

    return padded_signal, sr


def extract_mfcc_segments(segment, sr, num_mfcc=13, n_fft=2048, hop_length=512, num_segments=10):
    samples_per_segment = int(len(segment) / num_segments)
    num_mfcc_vectors_per_segment = math.ceil(samples_per_segment / hop_length)
    print(num_mfcc_vectors_per_segment)

    mfccs = []
    for d in range(num_segments):
        start = samples_per_segment * d
        finish = start + samples_per_segment

        mfcc = librosa.feature.mfcc(
            y=segment[start:finish],
            sr=sr,
            n_fft=n_fft,
            hop_length=hop_length,
            n_mfcc=num_mfcc
        )
        mfcc = mfcc.T
        if mfcc.shape[0] == num_mfcc_vectors_per_segment:
            mfccs.append(mfcc)

    return np.array(mfccs)
    

audio_path = path + "/test_tracks/sample9.wav"
segments, sr = load_and_pad_audio(audio_path, sample_rate=22500, target_duration=30)

all_mfcc_segments = []

for segment in segments:
    mfcc_segments = extract_mfcc_segments(segment, sr)
    all_mfcc_segments.extend(mfcc_segments)

segment_predictions = []
for mfcc_segment in all_mfcc_segments:
    X = mfcc_segment[np.newaxis, ...]
    prediction = model.predict(X)
    segment_predictions.append(prediction[0])

average_prediction = np.mean(segment_predictions, axis=0)

genre_dict = {
    0: "blues", 1: "classical", 2: "country", 3: "disco", 4: "hiphop",
    5: "jazz", 6: "metal", 7: "pop", 8: "reggae", 9: "rock"
}

for i, percent in enumerate(average_prediction * 100):
    print(f"{genre_dict[i]:<10}: {percent:.2f}%")

predicted_index = np.argmax(average_prediction)
print("\n🎵 Predicted Genre:", genre_dict[int(predicted_index)])


132
132
132
132
132
132
132
132
132
132
132
132
132
132
132
132
132
132
132
132
1/1 [==============================] - 0s 15ms/step
[0.04985958 0.6636276  0.05115266 0.01184815 0.01126793 0.16918129
 0.00566806 0.00662442 0.01743058 0.01333983]
blues     : 4.99%
classical : 66.36%
country   : 5.12%
disco     : 1.18%
hiphop    : 1.13%
jazz      : 16.92%
metal     : 0.57%
pop       : 0.66%
reggae    : 1.74%
rock      : 1.33%

🎵 Predicted Genre: classical
